# Composición Multivista

Cuando visualizamos una serie de datos con distintos campos de información, podríamos estar tentados a utilizar tantos canales de codificación visual como podamos: `x`, `y`, `color`, `size`, `shape`, y así sucesivamente. Sin embargo, a medida que aumenta el número de canales de codificación, un gráfico puede volverse rápidamente desordenado y difícil de leer. Una alternativa a la "sobrecarga" de un único gráfico es _componer múltiples gráficos (compose multiple charts)_ de manera que se faciliten las comparaciones rápidas.

En este *notebook*, examinaremos una variedad de operaciones para _composición multivista (multi-view composition)_:

- _layer_: colocar las gráficas compatibles directamente una encima de la otra,
- _facet_: dividir los datos en múltiples gráficos, organizados en filas o columnas,
- _concatenate_: posicionar gráficas arbitrarias dentro de un diseño compartido, y
- _repeat_: tomar una especificación de una gráfica base y aplicarla a múltiples campos de datos.

Luego veremos cómo estas operaciones forman un _álgebra de composición de visualización_, en la que las operaciones pueden combinarse para construir una variedad de despliegues de múltiples vistas.

_Este notebook es una traduccion y modificación del notebook **"Introduction to Vega-Lite / Altair"** , el cual es parte de [data visualization curriculum](https://github.com/uwdata/visualization-curriculum)._

_Todos los notebooks en español se encuentran en: [Altair en Simplificando Datos](https://github.com/SimplificandoDatos/Altair)_

In [1]:
import pandas as pd

import altair as alt

## Datos del tiempo (meteorológico)

Visualizaremos estadísticas del tiempo para las ciudades de Settle y New York en E.E.U.U. Carguemos los datos y demos un vistaso de las primeras diez filas:

In [2]:
weather = 'https://vega.github.io/vega-datasets/data/weather.csv'

In [3]:
df = pd.read_csv(weather)
df.head(10)

,location,date,precipitation,temp_max,temp_min,wind,weather
0,Seattle,2012-01-01,0.0,12.8,5.0,4.7,drizzle
1,Seattle,2012-01-02,10.9,10.6,2.8,4.5,rain
2,Seattle,2012-01-03,0.8,11.7,7.2,2.3,rain
3,Seattle,2012-01-04,20.3,12.2,5.6,4.7,rain
4,Seattle,2012-01-05,1.3,8.9,2.8,6.1,rain
5,Seattle,2012-01-06,2.5,4.4,2.2,2.2,rain
6,Seattle,2012-01-07,0.0,7.2,2.8,2.3,rain
7,Seattle,2012-01-08,0.0,10.0,2.8,2.0,sun
8,Seattle,2012-01-09,4.3,9.4,5.0,3.4,rain
9,Seattle,2012-01-10,1.0,6.1,0.6,3.4,rain


In [4]:
df.tail(10)

,location,date,precipitation,temp_max,temp_min,wind,weather
2912,New York,2015-12-22,4.8,15.6,11.1,3.8,fog
2913,New York,2015-12-23,29.5,17.2,8.9,4.5,fog
2914,New York,2015-12-24,0.5,20.6,13.9,4.9,fog
2915,New York,2015-12-25,2.5,17.8,11.1,0.9,fog
2916,New York,2015-12-26,0.3,15.6,9.4,4.8,drizzle
2917,New York,2015-12-27,2.0,17.2,8.9,5.5,fog
2918,New York,2015-12-28,1.3,8.9,1.7,6.3,snow
2919,New York,2015-12-29,16.8,9.4,1.1,5.3,fog
2920,New York,2015-12-30,9.4,10.6,5.0,3.0,fog
2921,New York,2015-12-31,1.5,11.1,6.1,5.5,fog


Crearemos despliegues multivista para examinar el tiempo en y entre las ciudades.

## Layer (Capa)

Una de las formas más comunes de combinar varios gráficos es colocar los marcadores en *capas (layers)* una encima de la otra. Si los dominios de escala subyacentes son compatibles, podemos fusionarlos para formar _ejes compartidos (shared axes)_. Si cualquiera de las codificaciones `x` o `y` no es compatible, podríamos crear un _gráfico de eje dual (dual-axis chart)_, que superpone los marcadores usando escalas y ejes separados.

### Shared Axes (Ejes compartidos)

Comencemos graficando el promedio de la temperatura mínima y máxima por mes:

In [5]:
alt.Chart(weather).mark_area().encode(
  alt.X('month(date):T',
  title= 'Mes'),
  alt.Y('average(temp_max):Q',
  title = 'Promedio de la temperatura máxima y mínima'),
  alt.Y2('average(temp_min):Q')
)

<VegaLite 3 object>

If you see this message, it means the renderer has not been properly enabled
for the frontend that you are using. For more information, see
https://altair-viz.github.io/user_guide/troubleshooting.html


_La gráfica nos muestra los rangos de temperatura para cada mes en la totalidad de nuestros datos. ¡Sin embargo, esto es bastante impreciso ya que combina los datos tanto de Seattle como de New York!_

Separemos los datos por ciudad usando el color como codificador, mientras que también ajustamos la opacidad del marcador para distinguir las áreas que se superponen:

In [6]:
alt.Chart(weather).mark_area(opacity=0.3).encode(
  alt.X('month(date):T',
  title='Mes'),
  alt.Y('average(temp_max):Q',
  title = 'Promedio de la temperatura máxima y mínima'),
  alt.Y2('average(temp_min):Q'),
  alt.Color('location:N',
  title='Ciudad')
)

<VegaLite 3 object>

If you see this message, it means the renderer has not been properly enabled
for the frontend that you are using. For more information, see
https://altair-viz.github.io/user_guide/troubleshooting.html


_Vemos que Seattle es más templado: más cálido en invierno y más fresco en verano._

En este caso hemos creado un gráfico por capas sin ninguna característica especial, simplemente subdividiendo los marcadores de área por color. Mientras que la gráfica de arriba nos muestra los rangos de temperatura, es posible que también queramos enfatizar la mitad del rango.

Vamos a crear un gráfico de líneas que muestre el punto medio de la temperatura media. Usaremos una transformación `calculate` para calcular los puntos medios entre las temperaturas mínimas y máximas diarias:

In [7]:
alt.Chart(weather).mark_line().transform_calculate(
  temp_mid='(+datum.temp_min + +datum.temp_max) / 2'
).encode(
  alt.X('month(date):T',
  title='Mes'),
  alt.Y('average(temp_mid):Q',
  title = 'Promedio del punto medio de la temperatura'),
  alt.Color('location:N',
  title='Ciudad')
)

<VegaLite 3 object>

If you see this message, it means the renderer has not been properly enabled
for the frontend that you are using. For more information, see
https://altair-viz.github.io/user_guide/troubleshooting.html


_Nota_: observe el uso de `+datum.temp_min` dentro de la transformación de cálculo. Como estamos cargando los datos directamente desde un archivo CSV sin instrucciones especiales de análisis, los valores de temperatura pueden estar representados internamente como texto. Agregar el `+` delante del valor obliga a que sea tratado como un número.

Ahora nos gustaría combinar estos gráficos superponiendo las líneas del punto medio sobre las áreas de rango. Usando la sintaxis `chart1 + chart2`, podemos especificar que queremos un nuevo gráfico en capas en el que `chart1` es la primera capa y `chart2` es una segunda capa dibujada encima:

In [8]:
tempMinMax = alt.Chart(weather).mark_area(opacity=0.3).encode(
  alt.X('month(date):T',
  title='Mes'),
  alt.Y('average(temp_max):Q',
  title = 'Promedio de la temperatura máxima y mínima'),
  alt.Y2('average(temp_min):Q'),
  alt.Color('location:N',
  title='Ciudad')
)

tempMid = alt.Chart(weather).mark_line().transform_calculate(
  temp_mid='(+datum.temp_min + +datum.temp_max) / 2'
).encode(
  alt.X('month(date):T'),
  alt.Y('average(temp_mid):Q',
  title = 'Promedio del punto medio de la temperatura'),
  alt.Color('location:N')
).properties(
  width=450,
  height=450
)

tempMinMax + tempMid

<VegaLite 3 object>

If you see this message, it means the renderer has not been properly enabled
for the frontend that you are using. For more information, see
https://altair-viz.github.io/user_guide/troubleshooting.html


_¡Ahora tenemos una gráfica de varias capas! Sin embargo, el título del eje `y` (aunque informativo) se ha vuelto un poco largo e inmanejable..._

Vamos a personalizar nuestros ejes para limpiar la gráfica. Si establecemos un título de eje personalizado dentro de una de las capas, se utilizará automáticamente como título de eje compartido para todas las capas:

In [9]:
tempMinMax = alt.Chart(weather).mark_area(opacity=0.3).encode(
  alt.X('month(date):T', title=None, axis=alt.Axis(format='%b')),
  alt.Y('average(temp_max):Q', title='Temperatura promedio °C'),
  alt.Y2('average(temp_min):Q'),
  alt.Color('location:N',
  title='Ciudad')
)

tempMid = alt.Chart(weather).mark_line().transform_calculate(
  temp_mid='(+datum.temp_min + +datum.temp_max) / 2'
).encode(
  alt.X('month(date):T'),
  alt.Y('average(temp_mid):Q'),
  alt.Color('location:N')
)

tempMinMax + tempMid

<VegaLite 3 object>

If you see this message, it means the renderer has not been properly enabled
for the frontend that you are using. For more information, see
https://altair-viz.github.io/user_guide/troubleshooting.html


Arriba usamos el operador `+`, un atajo conveniente para el método `layer` de Altair. Podemos generar una gráfica identica usando el método `layer` directamente:

In [10]:
alt.layer(tempMinMax, tempMid)

<VegaLite 3 object>

If you see this message, it means the renderer has not been properly enabled
for the frontend that you are using. For more information, see
https://altair-viz.github.io/user_guide/troubleshooting.html


Ten presente que el orden de las entradas de las capas es importante, ya que las capas subsiguientes se dibujarán sobre las capas anteriores. _Intenta cambiar el orden de los gráficos en las celdas de arriba. ¿Qué es lo que pasa? (Sugerencia: mira de cerca el color de los marcadores `line`)_.

### Dual-Axis Charts (Gráficas de doble eje)

_Seattle tiene reputación de una ciudad lluviosa. ¿Es merecida la reputación?_

Miremos la precipitación junto con la temperatura para aprender más. Primero creemos una gráfica base para mostrar la precipitación promedio mensual en Seattle:

In [11]:
alt.Chart(weather).transform_filter(
  'datum.location == "Seattle"'
).mark_line(
  interpolate='monotone',
  stroke='grey'
).encode(
  alt.X('month(date):T', title=None),
  alt.Y('average(precipitation):Q', title='Precipitación')
)

<VegaLite 3 object>

If you see this message, it means the renderer has not been properly enabled
for the frontend that you are using. For more information, see
https://altair-viz.github.io/user_guide/troubleshooting.html


Para facilitar la comparación con los datos de temperatura, creemos una nueva gráfica en capas. Aquí se muestra lo que sucede si tratamos de construir la gráfica en capas como lo hicimos anteriormente:

In [12]:
tempMinMax = alt.Chart(weather).transform_filter(
  'datum.location == "Seattle"'
).mark_area(opacity=0.3).encode(
  alt.X('month(date):T', title=None, axis=alt.Axis(format='%b')),
  alt.Y('average(temp_max):Q', title='Temperatura promedio °C'),
  alt.Y2('average(temp_min):Q')
)

precip = alt.Chart(weather).transform_filter(
  'datum.location == "Seattle"'
).mark_line(
  interpolate='monotone',
  stroke='grey'
).encode(
  alt.X('month(date):T'),
  alt.Y('average(precipitation):Q', title='Precipitación')
)

alt.layer(tempMinMax, precip)

<VegaLite 3 object>

If you see this message, it means the renderer has not been properly enabled
for the frontend that you are using. For more information, see
https://altair-viz.github.io/user_guide/troubleshooting.html


_¡Los valores de la precipitación usan un rango mucho menor del eje `y` que la temperatura!_

Por defecto, las gráficas por capas usan un *dominio compartido*: Los valores para el eje `x` o el `y` se combinan en todas las capas para determinar un rango compartido. Este comportamiento por defecto asume que las capas tienen las mismas unidades ¡Sin embargo, esto no es cierto en nuestro ejemplo, dado que estamos combinando valores de temperatura (grados Celsius) con valores de precipitación (pulgadas)!

Si queremos usar una escala diferente para el eje `y`,  necesitamos especificar cómo queremos que Altair *resuelva* los datos a través de las capas. En este caso, queremos resolver que los dominios de `scale` del eje `y` sean independientes (`independent`) en lugar de usar un dominio compartido (`shared`). El objeto `Chart` producido por un operador de capas incluye un método `resolve_scale` con el que podemos especificar la resolución deseada:

In [13]:
tempMinMax = alt.Chart(weather).transform_filter(
  'datum.location == "Seattle"'
).mark_area(opacity=0.3).encode(
  alt.X('month(date):T', title=None, axis=alt.Axis(format='%b')),
  alt.Y('average(temp_max):Q', title='Temperatura promedio °C'),
  alt.Y2('average(temp_min):Q')
)

precip = alt.Chart(weather).transform_filter(
  'datum.location == "Seattle"'
).mark_line(
  interpolate='monotone',
  stroke='grey'
).encode(
  alt.X('month(date):T'),
  alt.Y('average(precipitation):Q', title='Precipitación')
)

alt.layer(tempMinMax, precip).resolve_scale(y='independent')

<VegaLite 3 object>

If you see this message, it means the renderer has not been properly enabled
for the frontend that you are using. For more information, see
https://altair-viz.github.io/user_guide/troubleshooting.html


_Ahora podemos ver que el otoño es la estación más lluviosa de Seattle (con un pico en noviembre), complementada por veranos secos._

Puede que hayas notado alguna redundancia en nuestras especificaciones de la gráfica anterior: ambas utilizan el mismo conjunto de datos y el mismo filtro para mirar sólo a Seattle. Si lo deseas, puedes simplificar un poco el código proporcionando la transformación de datos y filtros al gráfico en las capas de nivel superior. Las capas individuales heredarán los datos si no tienen sus propias definiciones de datos:

In [14]:
tempMinMax = alt.Chart().mark_area(opacity=0.3).encode(
  alt.X('month(date):T', title=None, axis=alt.Axis(format='%b')),
  alt.Y('average(temp_max):Q', title='Temperatura promedio °C'),
  alt.Y2('average(temp_min):Q')
)

precip = alt.Chart().mark_line(
  interpolate='monotone',
  stroke='grey'
).encode(
  alt.X('month(date):T'),
  alt.Y('average(precipitation):Q', title='Precipitación')
)

alt.layer(tempMinMax, precip, data=weather).transform_filter(
  'datum.location == "Seattle"'
).resolve_scale(y='independent')

<VegaLite 3 object>

If you see this message, it means the renderer has not been properly enabled
for the frontend that you are using. For more information, see
https://altair-viz.github.io/user_guide/troubleshooting.html


Si bien los gráficos de doble eje pueden ser útiles, _a menudo son propensos a interpretaciones erróneas_, ya que las diferentes unidades y escalas de los ejes pueden ser incoherentes. Es factible considerar transformaciones que asignan diferentes campos de datos a unidades compartidas, por ejemplo mostrando [cuantiles](https://es.wikipedia.org/wiki/Cuantil) o cambio porcentual relativo.

## Facet (Facetas)

*Faceting* implica subdividir un conjunto de datos en grupos y crear una gráfica para cada grupo. En *notebooks* anteriores, aprendimos como crear este tipo de gráficas usando los canales de codificación `row` y `column`. Primero revisaremos estos canales y luego mostraremos como estos son un apartado de el operador `facet` que es más general.

Iniciemos con un histograma básico de los valores máximos de temperatura en Seattle:

In [15]:
alt.Chart(weather).mark_bar().transform_filter(
  'datum.location == "Seattle"'
).encode(
  alt.X('temp_max:Q', bin=True, title='Temperatura (°C)'),
  alt.Y('count():Q',
  title='Número de registros')
)

<VegaLite 3 object>

If you see this message, it means the renderer has not been properly enabled
for the frontend that you are using. For more information, see
https://altair-viz.github.io/user_guide/troubleshooting.html


_¿Cómo cambia este perfil de la temperatura basado en el tiempo de un día en particular - es decir, si hubo llovizna, niebla, lluvia, nieve o sol?_

Usemos el canal de codificación `column` para separar los datos por el tipo de tiempo. También, podemos usar `color` como un codificador redundante, usando rangos de color personalizados:

In [16]:
colors = alt.Scale(
  domain=['drizzle', 'fog', 'rain', 'snow', 'sun'],
  range=['#aec7e8', '#c7c7c7', '#1f77b4', '#9467bd', '#e7ba52']
)

alt.Chart(weather).mark_bar().transform_filter(
  'datum.location == "Seattle"'
).encode(
  alt.X('temp_max:Q', bin=True, title='Temperatura (°C)'),
  alt.Y('count():Q',
  title='Número de registros'),
  alt.Color('weather:N', scale=colors,
  title='Tiempo'),
  alt.Column('weather:N',
  title='Tiempo')
).properties(
  width=150,
  height=150
)

<VegaLite 3 object>

If you see this message, it means the renderer has not been properly enabled
for the frontend that you are using. For more information, see
https://altair-viz.github.io/user_guide/troubleshooting.html


_No es de extrañar que esos raros días de nieve se centren en las temperaturas más frías, seguidos de días lluviosos y nebulosos. Los días soleados son más cálidos y a pesar de los estereotipos de Seattle, son los más abundantes. Aunque como cualquier persona de Seattle puede decirte, la llovizna ocasionalmente llega, sin importar la temperatura._

Además de los canales de codificación `row` y `column` *adentro* de la definición de la gráfica, podemos tomar una definición básica de la gráfica y aplicar *faceting* usando el operador explícito `facet`.

Vamos a recrear la gráfica anterior, pero esta vez usando `facet`. Iniciamos con la misma definición del histograma básico, pero quitamos la fuente de los datos, la transformación de filtro y el canal de la columna. Entonces podemos invocar el método `facet`, pasando los datos y especificando que debemos separarlos en columnas según el campo `weather`. El método `facet` acepta ambos argumentos, `row` y `column`. Los dos pueden utilizarse juntos para crear una gráfica 2D con facetas.

Finalmente incluímos nuestra transformación de filtro, aplicandolo  en la gráfica *faceted* de nivel superior. Mientras que podríamos aplicar la transformación de filtro a la definición del histograma como lo hicimos antes, eso es menos eficiente. En lugar de filtrar los valores de "New York" dentro de cada celda de la faceta, aplicando el filtro al objeto Chart ya separado permite a Vega-Lite saber que podemos filtrar esos valores, antes de realizar la subdivisión.

In [17]:
colors = alt.Scale(
  domain=['drizzle', 'fog', 'rain', 'snow', 'sun'],
  range=['#aec7e8', '#c7c7c7', '#1f77b4', '#9467bd', '#e7ba52']
)

alt.Chart().mark_bar().encode(
  alt.X('temp_max:Q', bin=True, title='Temperatura (°C)'),
  alt.Y('count():Q',
  title='Número de registros'),
  alt.Color('weather:N', scale=colors,
  title='Tiempo')
).properties(
  width=150,
  height=150
).facet(
  data=weather,
  column='weather:N'
).transform_filter(
  'datum.location == "Seattle"'
)

<VegaLite 3 object>

If you see this message, it means the renderer has not been properly enabled
for the frontend that you are using. For more information, see
https://altair-viz.github.io/user_guide/troubleshooting.html


***Nota de Simplificando Datos:*** Con el método anterior, no pude cambiar el título por defecto de la gráfica para ponerlo en español, así que usé otro método para lograrlo, como se muestra a continuación.


In [18]:
colors = alt.Scale(
  domain=['drizzle', 'fog', 'rain', 'snow', 'sun'],
  range=['#aec7e8', '#c7c7c7', '#1f77b4', '#9467bd', '#e7ba52']
)

alt.Chart().mark_bar().encode(
  alt.X('temp_max:Q', bin=True, title='Temperatura (°C)'),
  alt.Y('count():Q',
  title='Número de registros'),
  alt.Color('weather:N', scale=colors,
  title='Tiempo')
).properties(
  width=150,
  height=150
).facet(
  data=weather,
  facet=alt.Facet('weather:N', title='Tiempo'),
  title='Tiempo en Seattle'
).transform_filter(
  'datum.location == "Seattle"'
)

<VegaLite 3 object>

If you see this message, it means the renderer has not been properly enabled
for the frontend that you are using. For more information, see
https://altair-viz.github.io/user_guide/troubleshooting.html


Dado todo el código extra anterior, ¿por qué querríamos usar un operador `facet` explícito? Para los gráficos básicos, sin duda deberíamos usar los canales de codificación de `columna` o `filas`, si podemos. Sin embargo, usar el operador `facet` explícitamente es útil si queremos *facetar* vistas compuestas, tales como gráficos por capas.

Volvamos a nuestros gráficos por capas de temperatura de antes. En lugar de graficar los datos de Nueva York y Seattle en el mismo gráfico, dividámoslos en facetas separadas. Las definiciones individuales de los gráficos son casi las mismas que antes: un gráfico de área y un gráfico de líneas. La única diferencia es que esta vez no pasaremos los datos directamente a los constructores de las gráficas; esperaremos y se los pasaremos al operador de facetas más tarde. Podemos colocar los gráficos en capas como antes, luego invocar `facet` en el objeto de gráfico en capas, pasando los datos y especificando las facetas por `column` basadas en el campo `location`:

In [19]:
tempMinMax = alt.Chart().mark_area(opacity=0.3).encode(
  alt.X('month(date):T', title=None, axis=alt.Axis(format='%b')),
  alt.Y('average(temp_max):Q', title='Temperatura promedio °C'),
  alt.Y2('average(temp_min):Q'),
  alt.Color('location:N')
)

tempMid = alt.Chart().mark_line().transform_calculate(
  temp_mid='(+datum.temp_min + +datum.temp_max) / 2'
).encode(
  alt.X('month(date):T'),
  alt.Y('average(temp_mid):Q'),
  alt.Color('location:N',
  title='Ciudad')
)

alt.layer(tempMinMax, tempMid).facet(
  data=weather,
  column='location:N',
)

<VegaLite 3 object>

If you see this message, it means the renderer has not been properly enabled
for the frontend that you are using. For more information, see
https://altair-viz.github.io/user_guide/troubleshooting.html


***Nota de Simplificando Datos:*** También construimos el gráfico anterior de otra manera para colocar el título en español.
**En las próximas gráficas ya no haremos este cambio, pero ya sabes que hacer si lo necesitas.**

In [20]:
tempMinMax = alt.Chart().mark_area(opacity=0.3).encode(
  alt.X('month(date):T', title=None, axis=alt.Axis(format='%b')),
  alt.Y('average(temp_max):Q', title='Temperatura promedio °C'),
  alt.Y2('average(temp_min):Q'),
  alt.Color('location:N')
)

tempMid = alt.Chart().mark_line().transform_calculate(
  temp_mid='(+datum.temp_min + +datum.temp_max) / 2'
).encode(
  alt.X('month(date):T'),
  alt.Y('average(temp_mid):Q'),
  alt.Color('location:N',
  title='Ciudad')
)

alt.layer(tempMinMax, tempMid).facet(
  data=weather,
  facet=alt.Facet('location:N', title='Ciudad')
)

<VegaLite 3 object>

If you see this message, it means the renderer has not been properly enabled
for the frontend that you are using. For more information, see
https://altair-viz.github.io/user_guide/troubleshooting.html


Las gráficas facetadas que hemos visto hasta ahora usan los mismos dominios de escala de ejes. Este valor predeterminado de usar escalas y ejes *compartidos* ayuda a una comparación precisa de los valores. Sin embargo, en algunos casos es posible que desees escalar cada gráfico de forma independiente, por ejemplo, si el rango de valores en las celdas difiere significativamente.

Similar a las gráficas en capas, las gráficas facetadas también soportan _resolving_ a escalas o ejes independientes a través de las gráficas. Veamos qué pasa si llamamos al método `resolve_axis` para solicitar que los ejes-Y sean *independientes* (`independent`):

In [21]:
tempMinMax = alt.Chart().mark_area(opacity=0.3).encode(
  alt.X('month(date):T', title=None, axis=alt.Axis(format='%b')),
  alt.Y('average(temp_max):Q', title='Temperatura promedio °C'),
  alt.Y2('average(temp_min):Q'),
  alt.Color('location:N',
  title='Ciudad')
)

tempMid = alt.Chart().mark_line().transform_calculate(
  temp_mid='(+datum.temp_min + +datum.temp_max) / 2'
).encode(
  alt.X('month(date):T'),
  alt.Y('average(temp_mid):Q'),
  alt.Color('location:N')
)

alt.layer(tempMinMax, tempMid).facet(
  data=weather,
  column='location:N'
).resolve_axis(y='independent')

<VegaLite 3 object>

If you see this message, it means the renderer has not been properly enabled
for the frontend that you are using. For more information, see
https://altair-viz.github.io/user_guide/troubleshooting.html


_La gráfica de arriba parece no haber cambiado, pero la gráfica de Seattle ahora incluye su propio eje._

¿Qué tal si en su lugar llamamos `resolve_scale` para resolver los dominios de escala que hay debajo?

In [22]:
tempMinMax = alt.Chart().mark_area(opacity=0.3).encode(
  alt.X('month(date):T', title=None, axis=alt.Axis(format='%b')),
  alt.Y('average(temp_max):Q', title='Temperatura promedio °C'),
  alt.Y2('average(temp_min):Q'),
  alt.Color('location:N',
  title='Ciudad')
)

tempMid = alt.Chart().mark_line().transform_calculate(
  temp_mid='(+datum.temp_min + +datum.temp_max) / 2'
).encode(
  alt.X('month(date):T'),
  alt.Y('average(temp_mid):Q'),
  alt.Color('location:N')
)

alt.layer(tempMinMax, tempMid).facet(
  data=weather,
  column='location:N'
).resolve_scale(y='independent')

<VegaLite 3 object>

If you see this message, it means the renderer has not been properly enabled
for the frontend that you are using. For more information, see
https://altair-viz.github.io/user_guide/troubleshooting.html


_Ahora vemos cada gráfica con diferentes dominios de escala de ejes. ¡En este caso, el uso de escalas independientes parece una mala idea! Los dominios no son muy diferentes, y uno podría pensar que Nueva York y Seattle tienen temperaturas máximas similares en verano._

Para tomar prestado un cliché: sólo porque *puedes* hacer algo, no significa que *deberías*...

## Concatenate (Concatenar)

Con *facet* se crean gráficos [small multiple](https://en.wikipedia.org/wiki/Small_multiple)  que muestran subdivisiones separadas de los datos. Sin embargo, es posible que deseemos crear un despliegue multivista con diferentes vistas del mismo conjunto de datos (no subconjuntos) o vistas que incluyan conjuntos de datos *diferentes*.

Altair proporciona operadores de *concatenación* para combinar gráficas arbitrarios en una gráfica compuesta. El operador `hconcat` (`|`) realiza la concatenación horizontal, mientras que el operador ``vconcat` (`&`) realiza la concatenación vertical.

Iniciemos con un auna gráfica de línea básica, mostrando el promedio de la temperatura máxima por mes, tanto para New York como de Seattle, muy parecidas a las vistas anteriormente:

In [23]:
alt.Chart(weather).mark_line().encode(
  alt.X('month(date):T', title=None),
  alt.Y('average(temp_max):Q', title = 'Temperatura promedio máxima'),
  alt.Color('location:N',
  title='Ciudad')
)

<VegaLite 3 object>

If you see this message, it means the renderer has not been properly enabled
for the frontend that you are using. For more information, see
https://altair-viz.github.io/user_guide/troubleshooting.html


_¿Y si queremos comparar no solo la temperatura en el tiempo, sino también los niveles de precipitación y la velocidad del vienteo?_

Creemos una gráfica concatenada que consista en tres gráficas. Inciemos definiendo una gráfica "base" que contenga todos los aspectos que deben compartirse por las tres gráficas. Luego modifiquemos esta gráfica base para crear variantes personalizadas, con diferentes codificaciones del eje Y para los campos `temp_max`, `precipitación` y `wind`. Entonces podemos concatenarlos usando el operador *pipe* (`|`):

In [24]:
base = alt.Chart(weather).mark_line().encode(
  alt.X('month(date):T', title=None),
  alt.Color('location:N',
  title='Ciudad')
).properties(
  width=240,
  height=180
)

temp = base.encode(alt.Y('average(temp_max):Q', title='Promedio de temperatur máxima'))
precip = base.encode(alt.Y('average(precipitation):Q', title = 'Promedio de precipitación'))
wind = base.encode(alt.Y('average(wind):Q', title='Promedio de velocidad del viento'))

temp | precip | wind

<VegaLite 3 object>

If you see this message, it means the renderer has not been properly enabled
for the frontend that you are using. For more information, see
https://altair-viz.github.io/user_guide/troubleshooting.html


Alternativamente, podríamos usar el método más explícito `alt.hconcat()` en lugar del operador *pipe* `|`. _Intenta reescribir el código de arriba para usar `hconcat` en su lugar._

La concatenación vertical funciona de manera similar a la concatenación horizontal. _Usando el operador `&` (o método `alt.vconcat`), modifica el código para usar un orden vertical en lugar de uno horizontal._

Por último, ten en cuenta que la concatenación horizontal y vertical pueden combinarse. _¿Qué pasa si escribes algo como `(temp | precip) & wind`?_

Nota: Observa la importancia de esos paréntesis... ¿Qué pasa si los quitas? Ten en cuenta que estos operadores sobrecargados todavía están sujetos a [las reglas de precedencia de operadores de Python](https://docs.python.org/3/reference/expressions.html#operator-precedence), ¡por lo que la concatenación vertical con `&` tendrá prioridad sobre la concatenación horizontal con `|`!

Como veremos más adelante, los operadores de concatenación le permiten combinar todos y cada uno de los gráficos en un panel de control multivista.

## Repeat (Repetir)

Los operadores de concatenación anteriores son bastante generales, lo que permite componer gráficos arbitrarios. Sin embargo, el ejemplo anterior era todavía un poco verborreico: tenemos tres gráficos muy similares, pero tenemos que definirlos por separado y luego concatenarlos.

Para los casos en los que sólo una o dos variables están cambiando, el operador `repeat` proporciona un atajo conveniente para crear múltiples gráficos. Dada una especificación de *plantilla* con algunas variables libres, el operador `repeat` creará un gráfico por cada asignación especificada a esas variables.

Vamos a recrear nuestro ejemplo de concatenación anterior usando el operador `repeat`. El único aspecto que cambia a través de los gráficos es la elección del campo de datos para el canal de codificación `y`. Para crear una plantilla, podemos usar la *variable de repetición* `alt.repeat('column')` como nuestro campo del eje Y. Este código simplemente indica que queremos usar la variable asignada al repetidor `column`, que organiza las gráficas repetidas en una dirección horizontal. (Como el repetidor sólo proporciona el nombre del campo, tenemos que especificar el tipo de datos de campo por separado como `type='quantitative'`.)

Luego invocamos el método `repeat`, pasando los nombres de los campos de datos para cada columna:

In [25]:
alt.Chart(weather).mark_line().encode(
  alt.X('month(date):T',title=None),
  alt.Y(alt.repeat('column'), aggregate='average', type='quantitative'),
  color='location:N'
).properties(
  width=240,
  height=180
).repeat(
  column=['temp_max', 'precipitation', 'wind']
)

<VegaLite 3 object>

If you see this message, it means the renderer has not been properly enabled
for the frontend that you are using. For more information, see
https://altair-viz.github.io/user_guide/troubleshooting.html


**Nota de Simplificando Datos:**

Usando el método 'repeat' no encontré la forma de modificar el título del eje Y, tampoco el de la leyenda.
Si tú sabes, agradecería que me compartas la información.

La repetición es compatible tanto para las columnas como para las filas. _¿Qué pasa si modificas el código anterior para usar `row` en lugar de `column`?_

¡También podemos usar la repetición de `row`  y `column` juntas! Una visualización común para el análisis exploratorio de datos es la [matriz de diagrama de dispersión (o SPLOM)](https://es.wikipedia.org/wiki/Diagrama_de_dispersión). Dada una colección de variables a inspeccionar, un *SPLOM* proporciona una cuadrícula de todas las gráficas por pares de esas variables, lo que nos permite evaluar las potenciales asociaciones .

Usemos el operador `repeat` para crear un SPLOM para los campos `temp_max`, `precipitation` y `wind`. Primero creamos nuestra plantilla, con variables de repetición para los campos de datos de los ejes X e Y. Luego invocamos `repeat`, pasando la lista de los nombres de los campos a usar tanto para `row` como para la `column`. Altair generará entonces el [producto cruzado (o, producto cartesiano)](https://es.wikipedia.org/wiki/Producto_cartesiano) para crear el espacio completo de gráficos repetidos:

In [26]:
alt.Chart().mark_point(filled=True, size=15, opacity=0.5).encode(
  alt.X(alt.repeat('column'), type='quantitative'),
  alt.Y(alt.repeat('row'), type='quantitative')
).properties(
  width=150,
  height=150
).repeat(
  data=weather,
  row=['temp_max', 'precipitation', 'wind'],
  column=['wind', 'precipitation', 'temp_max']
).transform_filter(
  'datum.location == "Seattle"'
)

<VegaLite 3 object>

If you see this message, it means the renderer has not been properly enabled
for the frontend that you are using. For more information, see
https://altair-viz.github.io/user_guide/troubleshooting.html


_Observando estas gráficas, no parece haber una fuerte asociación entre la precipitación y el viento, aunque sí vemos que los eventos extremos de viento y precipitación ocurren en rangos de temperatura similares (~5-15° C). Sin embargo, esta observación no es particularmente sorprendente: si revisamos nuestro histograma al principio de la sección **Facet**, podemos ver claramente que los días con temperaturas máximas en el rango de 5-15° C son los más comunes._

*Modifica el código anterior para obtener una mejor comprensión de la repetición de gráficos. Intenta añadir otra variable (`temp_min`) al SPLOM. ¿Qué sucede si reorganizas el orden de los nombres de los campos en los parámetros `row` o `column` para el operador `repeat`?*

_Por último, para apreciar realmente lo que el operador de repetición proporciona, tómate un momento para imaginar cómo podrías recrear el SPLOM usando sólo `hconcat` y `vconcat`._

## A View Composition Algebra (Álgebra de composición de visualización)


Juntos, los operadores de composición `layer`, `facet`, `concat` y `repeat` forman un *álgebra de composición de visualización*: los distintos operadores pueden combinarse para construir una variedad de visualizaciones multivista.

Como ejemplo, comencemos con dos gráficos básicos: un histograma y una simple línea (un solo marcador `rule`) que muestra un promedio global.

In [27]:
basic1 = alt.Chart(weather).transform_filter(
  'datum.location == "Seattle"'
).mark_bar().encode(
  alt.X('month(date):O', title='Mes'),
  alt.Y('average(temp_max):Q', title='Promedio de Temp. máxima')
)

basic2 = alt.Chart(weather).transform_filter(
  'datum.location == "Seattle"'
).mark_rule(stroke='firebrick').encode(
  alt.Y('average(temp_max):Q', title='Promedio de Temp. máxima')
)

basic1 | basic2

<VegaLite 3 object>

If you see this message, it means the renderer has not been properly enabled
for the frontend that you are using. For more information, see
https://altair-viz.github.io/user_guide/troubleshooting.html


Ahora podemos combinar las dos gráficas usando un operador `layer` y luego `repeat` esta gráfica en capas para mostrar histogramas con los promedios superpuestos para múltiples campos:

In [28]:
alt.layer(
  alt.Chart().mark_bar().encode(
    alt.X('month(date):O', title='Mes'),
    alt.Y(alt.repeat('column'), aggregate='average', type='quantitative')
  ),
  alt.Chart().mark_rule(stroke='firebrick').encode(
    alt.Y(alt.repeat('column'), aggregate='average', type='quantitative')
  )
).properties(
  width=200,
  height=150
).repeat(
  data=weather,
  column=['temp_max', 'precipitation', 'wind']
).transform_filter(
  'datum.location == "Seattle"'
)

<VegaLite 3 object>

If you see this message, it means the renderer has not been properly enabled
for the frontend that you are using. For more information, see
https://altair-viz.github.io/user_guide/troubleshooting.html


Enfocándonos solo en los operadores de composición multivista, el modelo para la visualización anterior es:

```
repeat(column=[...])
|- layer
   |- basic1
   |- basic2
```

Ahora exploremos como podemos aplicar *todos* los operadores dentro de un [dashboard final](https://www.cyberclick.es/numerical-blog/que-es-un-dashboard) que provea una visión general de el tiempo de Seattle. Combinaremos el SPLOM y las facetas de los histogramas de las secciones anteriores con los histogramas repetidos de arriba.

*Una explicación de qué es un dashboard en inglés lo encuentras en este [enlace](https://en.wikipedia.org/wiki/Dashboard_%28business%29)*

In [29]:
splom = alt.Chart().mark_point(filled=True, size=15, opacity=0.5).encode(
  alt.X(alt.repeat('column'), type='quantitative'),
  alt.Y(alt.repeat('row'), type='quantitative')
).properties(
  width=125,
  height=125
).repeat(
  row=['temp_max', 'precipitation', 'wind'],
  column=['wind', 'precipitation', 'temp_max']
)

dateHist = alt.layer(
  alt.Chart().mark_bar().encode(
    alt.X('month(date):O', title='Mes'),
    alt.Y(alt.repeat('row'), aggregate='average', type='quantitative')
  ),
  alt.Chart().mark_rule(stroke='firebrick').encode(
    alt.Y(alt.repeat('row'), aggregate='average', type='quantitative')
  )
).properties(
  width=175,
  height=125
).repeat(
  row=['temp_max', 'precipitation', 'wind']
)

tempHist = alt.Chart(weather).mark_bar().encode(
  alt.X('temp_max:Q', bin=True, title='Temperatura (°C)'),
  alt.Y('count():Q'),
  alt.Color('weather:N', scale=alt.Scale(
    domain=['drizzle', 'fog', 'rain', 'snow', 'sun'],
    range=['#aec7e8', '#c7c7c7', '#1f77b4', '#9467bd', '#e7ba52']
  ))
).properties(
  width=115,
  height=100
).facet(
  column='weather:N'
)

alt.vconcat(
  alt.hconcat(splom, dateHist),
  tempHist,
  data=weather,
  title='Dashboard del tiempo de Seattle'
).transform_filter(
  'datum.location == "Seattle"'
).resolve_legend(
  color='independent'
).configure_axis(
  labelAngle=0
)

<VegaLite 3 object>

If you see this message, it means the renderer has not been properly enabled
for the frontend that you are using. For more information, see
https://altair-viz.github.io/user_guide/troubleshooting.html


El modelo completo de la composición para este *dashboard* es:

```
vconcat
|- hconcat
|  |- repeat(row=[...], column=[...])
|  |  |- splom base chart
|  |- repeat(row=[...])
|     |- layer
|        |- dateHist base chart 1
|        |- dateHist base chart 2
|- facet(column='weather')
   |- tempHist base chart
```

_¡Fiuuuuu!_ El *dashboard* también incluye unas pocas personalizaciones para mejorar el diseño:

- Ajustamos las propiedades de `width` y `height` del gráfico para ayudar a la alineación y asegurar que la visualización completa encaje en la pantalla.
- Añadimos `resolve_legend(color='independent')` para asegurar que la leyenda de los colores se asocie directamente con los histogramas coloreados de la temperatura. De lo contrario, la leyenda se resolverá en el tablero como un todo.
- Utilizamos `configure_axis(labelAngle=0)` para asegurarnos de que no se giran las etiquetas de los ejes. Esto ayuda a asegurar una alineación adecuada entre los diagramas de dispersión en el SPLOM y los histogramas por mes a la derecha.

_¡Intenta quitar o modificar cualquiera de estos ajustes y ve cómo responde el diseño del tablero!_

Este *dashboard* puede ser reutilizado para mostrar datos de otras ubicaciones o de otros conjuntos de datos. Actualiza el *dashboard* para mostrar los patrones del clima de New York en lugar de Seattle.

## Resumen

Para más detalles sobre la composición multivista, incluyendo el control sobre el espaciado de subgráficas y las etiquetas de encabezado, consulte la documentación [Altair Compound Charts](https://altair-viz.github.io/user_guide/compound_charts.html).

Ahora que hemos visto cómo componer múltiples vistas, estamos listos para ponerlas en acción. Además de la presentación estática de los datos, las vistas múltiples pueden permitir la exploración interactiva multidimensional. Por ejemplo, usando _selecciones enlazadas_ podemos resaltar puntos en una vista para ver los valores correspondientes resaltados en otras vistas.

En el siguiente *notebook*, examinaremos cómo crear *selecciones interactivas* tanto para composiciones individuales como para composiciones multivista.